In [ ]:
import warnings
warnings.filterwarnings('ignore')
import transect_viz
from transect_viz import transect_generator
from transect_viz import transect_animator
from transect_viz import transect_data
from transect_viz import transect_cdec_data
from transect_viz.transect_animator import GeneratedECMapAnimator
import panel as pn
pn.extension()

In [ ]:
# uncomment to generate equidistant points from line string geojson (one long line string i.e.)
#transect_generator.convert_geojson_line_to_pts('geodata/vns2dgl.geojson','geodata/vns2dgl_pts.geojson')
#transect_generator.convert_geojson_line_to_pts('geodata/middle_river.geojson','geodata/middle_river_pts.geojson')
#transect_generator.convert_geojson_line_to_pts('geodata/victoria_canal.geojson','geodata/victoria_canal_pts.geojson')
#transect_generator.convert_geojson_line_to_pts('geodata/paradise_cut.geojson','geodata/paradise_cut_pts.geojson')

In [ ]:
# all variables for driving the animation
# These are hardcoded here but can be moved to config file in the future.
sdate='2022-01-01'
edate='2022-11-10'
value_range=(300,800)
REGENERATE_CSV = True # set to True if sdate, edate are changed or data needs to be redownloaded from CDEC

In [ ]:
# -- Uncomment below only if data is not updated or available in the csv files
if (REGENERATE_CSV):
    stations_csv_file, data_csv_file = transect_cdec_data.generate_csv_files('fabian_tract', ['ORM', 'ODM', 'OLD', 'GLE', 'GLC', 'DGL'], 'ec', sdate, edate)
    print((stations_csv_file, data_csv_file))
    stations_csv_file, data_csv_file = transect_cdec_data.generate_csv_files('sugarcut', ['DAR', 'SUR', 'SGA'], 'ec', sdate, edate)
    print((stations_csv_file, data_csv_file))
    stations_csv_file, data_csv_file = transect_cdec_data.generate_csv_files('vns2dgl', ['SJR', 'MSD', 'OH1', 'UNI', 'ORX', 'DGL'], 'ec', sdate, edate)
    print((stations_csv_file, data_csv_file))
    stations_csv_file, data_csv_file = transect_cdec_data.generate_csv_files('middle_river', ['UNI', 'MRU', 'MHO', 'MTB', 'MRX'], 'ec', sdate, edate)
    print((stations_csv_file, data_csv_file))
    stations_csv_file, data_csv_file = transect_cdec_data.generate_csv_files('victoria_canal',['VIC', 'VCU', 'WCI', 'ORI'] , 'ec', sdate, edate)
    print((stations_csv_file, data_csv_file))
    stations_csv_file, data_csv_file = transect_cdec_data.generate_csv_files('paradise_cut', ['PDC'], 'ec', sdate, edate)
    print((stations_csv_file, data_csv_file))
    stations_csv_file, data_csv_file = transect_cdec_data.generate_csv_files('tom_paine_slough', ['TPI', 'TPP'], 'ec', sdate, edate)
    print((stations_csv_file, data_csv_file))
    #stations_csv_file, data_csv_file = transect_cdec_data.generate_csv_files('', , 'ec', sdate, edate)
    #print((stations_csv_file, data_csv_file))


In [ ]:
fabian = transect_animator.create_transect_map('../geodata/fabian_tract_pts.geojson', 'fabian_tract_stations.csv',
                             'fabian_tract_ec_data.csv', close_transect=True)
sugar_cut = transect_animator.create_transect_map('../geodata/sugarcut_pts.geojson',
                                          'sugarcut_stations.csv', 'sugarcut_ec_data.csv')
vns2dgl = transect_animator.create_transect_map('../geodata/vns2dgl_pts.geojson',
                                        'vns2dgl_stations.csv', 'vns2dgl_ec_data.csv')
midr = transect_animator.create_transect_map('../geodata/middle_river_pts.geojson',
                                     'middle_river_stations.csv', 'middle_river_ec_data.csv')
vic = transect_animator.create_transect_map('../geodata/victoria_canal_pts.geojson',
                                   'victoria_canal_stations.csv', 'victoria_canal_ec_data.csv')
pdc = transect_animator.create_transect_map('../geodata/paradise_cut_pts.geojson',
                                    'paradise_cut_stations.csv', 'paradise_cut_ec_data.csv')
tps = transect_animator.create_transect_map('../geodata/tom_paine_slough_pts.geojson',
                                    'tom_paine_slough_stations.csv', 'tom_paine_slough_ec_data.csv')
tmap_list = [fabian, sugar_cut, vns2dgl, midr, vic, pdc, tps]

In [ ]:
if REGENERATE_CSV:
    transect_cdec_data.generate_csv_files('south_delta',
                                          ['ORM', 'ODM', 'OLD', 'GLE', 'GLC', 'DAR', 'SGA', 'VNS', 'MSD',
                                              'OH1', 'ORX', 'MRU', 'MAB', 'VCU', 'WCI', 'ORI', 'PDC'],
                                          'flow', sdate, edate)

In [ ]:
flow_stations = transect_data.read_stations_csv_file('south_delta_stations.csv')
flow_data = transect_data.read_data_csv_file('south_delta_flow_data.csv')
stations_display_info = transect_data.read_station_display_info('../data/station_info_file.csv')
barrier_info = transect_data.read_barriers_info('../data/barriers.csv')

In [ ]:
pn.config.sizing_mode='scale_both'
map_animator = GeneratedECMapAnimator(tmap_list, sdate, edate, flow_stations, stations_display_info, flow_data, barrier_info)

app = map_animator.create_app()
#pn.serve(app, static_dirs={'transect_data':'./transect_data/'}) # uncomment only for running locally

#def load_anim():
_= map_animator.setup_main_panel()

#pn.state.onload(load_anim)

#app.show() # uncomment only if running notebook